# Chapter 2 - Exercise 5: Nutrient Diffusion and Cell Viability

## Understanding Mass Transport Limitations in Scaffold-Free Biofabrication

**Learning Objectives:**
- Model oxygen and nutrient gradients in spheroids and 3D constructs
- Understand formation of necrotic cores due to diffusion limitations
- Calculate critical size limits for scaffold-free constructs
- Explore how vascularization improves nutrient delivery

**Python Skills:**
- Solving differential equations (simplified diffusion models)
- Creating 2D contour plots and gradient visualizations
- Animating time-dependent processes

---

## Setup and Imports

Run this cell first to install required packages and import libraries.

In [ ]:
# Install required packages
!pip install numpy matplotlib scipy pandas seaborn ipywidgets

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("✅ All packages imported successfully!")
print("📚 Ready to explore nutrient diffusion in scaffold-free constructs!")

## 📖 Background: Diffusion Limitations in 3D Tissue Constructs

### The Diffusion Problem

In scaffold-free biofabrication, cells must rely on **diffusion** to receive oxygen and nutrients. This creates fundamental limitations:

- **Oxygen** has low solubility in aqueous media (~0.2 mM at 37°C)
- **Diffusion** is relatively slow over distances >100 μm
- **Cell consumption** creates gradients that worsen with construct size
- **Necrotic cores** form when oxygen drops below survival thresholds

### Mathematical Model

For a spherical construct, oxygen concentration C(r,t) follows:

**Fick's Second Law in Spherical Coordinates:**
```
∂C/∂t = D * (∂²C/∂r² + (2/r) * ∂C/∂r) - R_max * C/(K_m + C)
```

Where:
- D = diffusion coefficient (cm²/s)
- R_max = maximum consumption rate (mM/s)
- K_m = Michaelis-Menten constant (mM)
- C = oxygen concentration (mM)

### Critical Parameters

| Parameter | Typical Value | Units |
|-----------|---------------|-------|
| D_oxygen | 2.4 × 10⁻⁵ | cm²/s |
| R_max | 1.0 × 10⁻⁸ | mM/s |
| K_m | 0.01 | mM |
| C_critical | 0.01 | mM |
| C_surface | 0.2 | mM |

## 🗃️ Tissue Properties Database

Different cell types have varying oxygen consumption rates and critical thresholds.

In [ ]:
# Tissue properties database
tissue_properties = {
    'Hepatocytes': {
        'R_max': 2.5e-8,  # mM/s - high metabolism
        'K_m': 0.015,     # mM
        'C_critical': 0.02,  # mM - sensitive to hypoxia
        'cell_density': 1e8,  # cells/cm³
        'description': 'High oxygen consumption, sensitive to hypoxia'
    },
    'Cardiomyocytes': {
        'R_max': 2.0e-8,  # mM/s
        'K_m': 0.012,     # mM
        'C_critical': 0.015, # mM
        'cell_density': 8e7,  # cells/cm³
        'description': 'High energy demand, critical for heart function'
    },
    'Chondrocytes': {
        'R_max': 0.5e-8,  # mM/s - low metabolism
        'K_m': 0.008,     # mM
        'C_critical': 0.005, # mM - hypoxia tolerant
        'cell_density': 2e7,  # cells/cm³
        'description': 'Low oxygen consumption, hypoxia tolerant'
    },
    'Tumor_cells': {
        'R_max': 1.5e-8,  # mM/s
        'K_m': 0.010,     # mM
        'C_critical': 0.008, # mM - somewhat tolerant
        'cell_density': 1.2e8, # cells/cm³
        'description': 'Variable metabolism, can survive low oxygen'
    },
    'Neural_cells': {
        'R_max': 1.8e-8,  # mM/s
        'K_m': 0.020,     # mM
        'C_critical': 0.025, # mM - very sensitive
        'cell_density': 6e7,  # cells/cm³
        'description': 'High oxygen demand, extremely sensitive to hypoxia'
    },
    'Fibroblasts': {
        'R_max': 0.8e-8,  # mM/s - moderate metabolism
        'K_m': 0.009,     # mM
        'C_critical': 0.010, # mM
        'cell_density': 5e7,  # cells/cm³
        'description': 'Moderate consumption, relatively robust'
    }
}

# Physical constants
constants = {
    'D_oxygen': 2.4e-5,    # cm²/s - oxygen diffusion coefficient
    'C_surface': 0.2,      # mM - surface oxygen concentration
    'temp': 37,            # °C - physiological temperature
}

# Display tissue properties
print("🧬 Tissue Properties Database")
print("=" * 50)
df_tissues = pd.DataFrame(tissue_properties).T
display(df_tissues[['R_max', 'C_critical', 'cell_density', 'description']])

## 🧮 Diffusion Model Implementation

We'll implement a simplified 1D radial diffusion model for spherical constructs.

In [ ]:
class SpheroidDiffusionModel:
    def __init__(self, radius, tissue_type, n_points=50):
        """
        Initialize spheroid diffusion model
        
        Parameters:
        - radius: spheroid radius (cm)
        - tissue_type: cell type from database
        - n_points: number of radial grid points
        """
        self.radius = radius
        self.tissue_type = tissue_type
        self.n_points = n_points
        
        # Get tissue properties
        self.props = tissue_properties[tissue_type]
        
        # Create radial grid
        self.r = np.linspace(1e-6, radius, n_points)  # Avoid r=0
        self.dr = self.r[1] - self.r[0]
        
        # Initialize concentration profile
        self.C = np.full(n_points, constants['C_surface'])
        
    def consumption_rate(self, C):
        """
        Michaelis-Menten consumption kinetics
        """
        R_max = self.props['R_max']
        K_m = self.props['K_m']
        return R_max * C / (K_m + C)
    
    def steady_state_solution(self):
        """
        Solve for steady-state concentration profile
        """
        # Simplified analytical approach for low K_m
        # Using dimensionless variables
        
        # Thiele modulus
        phi_squared = (self.props['R_max'] * self.radius**2) / (constants['D_oxygen'] * constants['C_surface'])
        
        # Analytical solution for sphere with first-order kinetics
        phi = np.sqrt(phi_squared)
        
        C_profile = np.zeros_like(self.r)
        
        for i, r in enumerate(self.r):
            if r < self.radius:
                eta = r / self.radius
                if phi > 0.1:  # Significant consumption
                    C_profile[i] = constants['C_surface'] * (np.sinh(phi * eta) / (eta * np.sinh(phi)))
                else:  # Low consumption limit
                    C_profile[i] = constants['C_surface'] * (1 - phi_squared * (1 - eta**2) / 6)
            else:
                C_profile[i] = constants['C_surface']
        
        # Handle center point (r=0)
        if len(C_profile) > 0:
            C_profile[0] = C_profile[1]  # No flux at center
        
        return C_profile
    
    def calculate_viability_zone(self):
        """
        Calculate viable tissue volume based on oxygen threshold
        """
        C_profile = self.steady_state_solution()
        C_crit = self.props['C_critical']
        
        # Find viable radius
        viable_indices = C_profile >= C_crit
        
        if np.any(viable_indices):
            viable_radius = np.max(self.r[viable_indices])
            necrotic_radius = self.radius - viable_radius
        else:
            viable_radius = 0
            necrotic_radius = self.radius
        
        # Calculate volumes
        total_volume = (4/3) * np.pi * self.radius**3
        viable_volume = (4/3) * np.pi * viable_radius**3
        necrotic_volume = total_volume - viable_volume
        
        return {
            'C_profile': C_profile,
            'viable_radius': viable_radius,
            'necrotic_radius': necrotic_radius,
            'total_volume': total_volume,
            'viable_volume': viable_volume,
            'necrotic_volume': necrotic_volume,
            'viability_fraction': viable_volume / total_volume if total_volume > 0 else 0
        }

def find_critical_radius(tissue_type, viability_threshold=0.8):
    """
    Find maximum radius for given viability threshold
    """
    def objective(radius):
        model = SpheroidDiffusionModel(radius[0], tissue_type)
        result = model.calculate_viability_zone()
        return result['viability_fraction'] - viability_threshold
    
    try:
        # Search for critical radius
        result = fsolve(objective, [0.01])[0]  # Start search at 100 μm
        return max(0, result)
    except:
        return 0.01  # Default fallback

print("✅ Diffusion model implemented successfully!")

## 📊 Critical Size Analysis

Let's calculate critical radii for different tissue types and analyze size limitations.

In [ ]:
# Calculate critical radii for all tissue types
critical_data = []

viability_thresholds = [0.9, 0.8, 0.7, 0.5]

for tissue in tissue_properties.keys():
    for threshold in viability_thresholds:
        critical_radius = find_critical_radius(tissue, threshold)
        critical_data.append({
            'Tissue': tissue,
            'Viability_Threshold': threshold,
            'Critical_Radius_cm': critical_radius,
            'Critical_Radius_μm': critical_radius * 10000,
            'Critical_Diameter_μm': critical_radius * 20000
        })

df_critical = pd.DataFrame(critical_data)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Critical diameter vs viability threshold
for tissue in tissue_properties.keys():
    tissue_data = df_critical[df_critical['Tissue'] == tissue]
    ax1.plot(tissue_data['Viability_Threshold'], 
             tissue_data['Critical_Diameter_μm'], 
             marker='o', linewidth=2, label=tissue)

ax1.set_xlabel('Viability Threshold (fraction alive)')
ax1.set_ylabel('Critical Diameter (μm)')
ax1.set_title('Critical Spheroid Size vs Viability Threshold')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0.45, 0.95)

# Plot 2: Tissue comparison at 80% viability
threshold_80_data = df_critical[df_critical['Viability_Threshold'] == 0.8]
colors = plt.cm.Set3(np.linspace(0, 1, len(threshold_80_data)))

bars = ax2.bar(range(len(threshold_80_data)), 
               threshold_80_data['Critical_Diameter_μm'],
               color=colors)

ax2.set_xlabel('Tissue Type')
ax2.set_ylabel('Critical Diameter (μm)')
ax2.set_title('Critical Size Comparison (80% Viability)')
ax2.set_xticks(range(len(threshold_80_data)))
ax2.set_xticklabels(threshold_80_data['Tissue'], rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 10,
             f'{height:.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Display summary table
print("\n📋 Critical Size Summary (80% Viability)")
print("=" * 60)
summary_80 = threshold_80_data[['Tissue', 'Critical_Diameter_μm']].copy()
summary_80['Critical_Diameter_μm'] = summary_80['Critical_Diameter_μm'].round(0)
summary_80 = summary_80.sort_values('Critical_Diameter_μm', ascending=False)
display(summary_80)

print("\n💡 Key Insights:")
print(f"• Largest viable size: {summary_80.iloc[0]['Tissue']} ({summary_80.iloc[0]['Critical_Diameter_μm']:.0f} μm)")
print(f"• Smallest viable size: {summary_80.iloc[-1]['Tissue']} ({summary_80.iloc[-1]['Critical_Diameter_μm']:.0f} μm)")
print(f"• Size range spans {summary_80.iloc[0]['Critical_Diameter_μm']/summary_80.iloc[-1]['Critical_Diameter_μm']:.1f}x difference")

## 🎛️ Interactive Diffusion Simulator

Explore how different parameters affect oxygen gradients and cell viability.

In [ ]:
# Interactive widget function
def interactive_diffusion_plot(radius_um=400, tissue_type='Hepatocytes', show_zones=True):
    """
    Interactive visualization of oxygen diffusion in spheroids
    """
    # Convert radius to cm
    radius_cm = radius_um / 10000
    
    # Create model
    model = SpheroidDiffusionModel(radius_cm, tissue_type)
    result = model.calculate_viability_zone()
    
    # Create plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Concentration profile
    r_um = model.r * 10000  # Convert to μm
    C_profile = result['C_profile']
    C_crit = model.props['C_critical']
    
    ax1.plot(r_um, C_profile, 'b-', linewidth=3, label='Oxygen concentration')
    ax1.axhline(C_crit, color='red', linestyle='--', linewidth=2, 
                label=f'Critical threshold ({C_crit:.3f} mM)')
    
    if show_zones:
        viable_mask = C_profile >= C_crit
        ax1.fill_between(r_um, 0, C_profile, where=viable_mask, 
                        alpha=0.3, color='green', label='Viable zone')
        ax1.fill_between(r_um, 0, C_profile, where=~viable_mask, 
                        alpha=0.3, color='red', label='Necrotic zone')
    
    ax1.set_xlabel('Distance from center (μm)')
    ax1.set_ylabel('Oxygen concentration (mM)')
    ax1.set_title(f'Oxygen Profile - {tissue_type}\nRadius: {radius_um} μm')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, radius_um)
    ax1.set_ylim(0, constants['C_surface'] * 1.1)
    
    # Plot 2: Cross-section visualization
    viable_radius_um = result['viable_radius'] * 10000
    
    # Draw spheroid cross-section
    circle_outer = plt.Circle((0, 0), radius_um, fill=False, 
                             edgecolor='black', linewidth=2)
    
    if viable_radius_um > 0:
        circle_viable = plt.Circle((0, 0), viable_radius_um, 
                                  color='lightgreen', alpha=0.7, label='Viable tissue')
        ax2.add_patch(circle_viable)
    
    if viable_radius_um < radius_um:
        # Necrotic core
        if viable_radius_um > 0:
            necrotic_ring = plt.Circle((0, 0), radius_um, 
                                     color='lightcoral', alpha=0.7)
            ax2.add_patch(necrotic_ring)
            # Re-add viable zone on top
            circle_viable = plt.Circle((0, 0), viable_radius_um, 
                                      color='lightgreen', alpha=0.7)
            ax2.add_patch(circle_viable)
        else:
            # Completely necrotic
            circle_necrotic = plt.Circle((0, 0), radius_um, 
                                       color='darkred', alpha=0.7, label='Necrotic tissue')
            ax2.add_patch(circle_necrotic)
    
    ax2.add_patch(circle_outer)
    ax2.set_xlim(-radius_um*1.2, radius_um*1.2)
    ax2.set_ylim(-radius_um*1.2, radius_um*1.2)
    ax2.set_aspect('equal')
    ax2.set_xlabel('Distance (μm)')
    ax2.set_ylabel('Distance (μm)')
    ax2.set_title('Spheroid Cross-Section')
    ax2.grid(True, alpha=0.3)
    
    # Add legend for cross-section
    if viable_radius_um > 0:
        ax2.scatter([], [], c='lightgreen', s=100, label='Viable tissue')
    if viable_radius_um < radius_um:
        ax2.scatter([], [], c='lightcoral', s=100, label='Necrotic tissue')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Display metrics
    print(f"\n📊 Analysis Results for {tissue_type}")
    print("=" * 50)
    print(f"Spheroid radius: {radius_um:.0f} μm")
    print(f"Viable radius: {viable_radius_um:.0f} μm")
    print(f"Necrotic core size: {(radius_um - viable_radius_um):.0f} μm")
    print(f"Viability fraction: {result['viability_fraction']:.1%}")
    print(f"Center oxygen: {C_profile[0]:.3f} mM")
    print(f"Surface oxygen: {C_profile[-1]:.3f} mM")
    
    if result['viability_fraction'] < 0.8:
        print("\n⚠️  Warning: Low viability! Consider reducing size or improving vascularization.")
    elif result['viability_fraction'] > 0.95:
        print("\n✅ Excellent viability! Size is well within diffusion limits.")
    else:
        print("\n✓ Good viability, but approaching diffusion limits.")

# Create interactive widgets
radius_slider = widgets.IntSlider(
    value=400,
    min=50,
    max=1000,
    step=50,
    description='Radius (μm):',
    style={'description_width': 'initial'}
)

tissue_dropdown = widgets.Dropdown(
    options=list(tissue_properties.keys()),
    value='Hepatocytes',
    description='Tissue type:',
    style={'description_width': 'initial'}
)

zones_checkbox = widgets.Checkbox(
    value=True,
    description='Show viable/necrotic zones',
    style={'description_width': 'initial'}
)

# Display interactive plot
interactive_plot = widgets.interactive(interactive_diffusion_plot,
                                     radius_um=radius_slider,
                                     tissue_type=tissue_dropdown,
                                     show_zones=zones_checkbox)

display(interactive_plot)

## 🩸 Vascularization Impact Analysis

Let's explore how blood vessels can overcome diffusion limitations.

In [ ]:
def simulate_vascularized_tissue(radius_cm, tissue_type, vessel_density=100):
    """
    Simulate tissue with blood vessels
    
    Parameters:
    - radius_cm: tissue radius in cm
    - tissue_type: cell type
    - vessel_density: vessels per cm³
    """
    # Assume vessels are uniformly distributed
    # Each vessel supplies a cylindrical region
    volume_per_vessel = 1 / vessel_density  # cm³
    supply_radius = (volume_per_vessel / (np.pi * (2 * radius_cm)))**(1/2)
    
    # Maximum diffusion distance is supply_radius
    max_diffusion_distance = min(supply_radius, 0.01)  # Cap at 100 μm
    
    # Calculate viable fraction based on vessel spacing
    props = tissue_properties[tissue_type]
    
    # Simple model: if diffusion distance < critical distance, full viability
    critical_distance = np.sqrt(constants['D_oxygen'] * constants['C_surface'] / props['R_max'])
    
    if max_diffusion_distance < critical_distance:
        viability_fraction = 1.0
    else:
        viability_fraction = critical_distance / max_diffusion_distance
    
    return {
        'supply_radius': supply_radius,
        'max_diffusion_distance': max_diffusion_distance,
        'viability_fraction': min(1.0, viability_fraction),
        'vessel_spacing': 2 * supply_radius
    }

# Compare avascular vs vascularized constructs
radii_um = np.array([200, 400, 600, 800, 1000, 1500, 2000])
radii_cm = radii_um / 10000

vessel_densities = [50, 100, 200, 500]  # vessels/cm³
tissue_type = 'Hepatocytes'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Viability vs size for different vascularization levels
# Avascular case
avascular_viability = []
for radius_cm in radii_cm:
    model = SpheroidDiffusionModel(radius_cm, tissue_type)
    result = model.calculate_viability_zone()
    avascular_viability.append(result['viability_fraction'])

ax1.plot(radii_um, avascular_viability, 'r-', linewidth=3, 
         marker='o', label='Avascular', markersize=8)

# Vascularized cases
colors = ['orange', 'green', 'blue', 'purple']
for i, density in enumerate(vessel_densities):
    vascular_viability = []
    for radius_cm in radii_cm:
        result = simulate_vascularized_tissue(radius_cm, tissue_type, density)
        vascular_viability.append(result['viability_fraction'])
    
    ax1.plot(radii_um, vascular_viability, color=colors[i], linewidth=2,
             marker='s', label=f'{density} vessels/cm³', markersize=6)

ax1.axhline(0.8, color='gray', linestyle='--', alpha=0.7, label='80% viability threshold')
ax1.set_xlabel('Construct Size (μm)')
ax1.set_ylabel('Viability Fraction')
ax1.set_title(f'Vascularization Impact - {tissue_type}')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.05)

# Plot 2: Maximum viable size vs vessel density
densities = np.logspace(1, 3, 20)  # 10 to 1000 vessels/cm³
max_sizes = []

for density in densities:
    # Find maximum size for 80% viability
    max_size = 0
    for test_radius in np.linspace(0.001, 0.3, 100):  # Up to 3 mm
        result = simulate_vascularized_tissue(test_radius, tissue_type, density)
        if result['viability_fraction'] >= 0.8:
            max_size = test_radius * 10000  # Convert to μm
    max_sizes.append(max_size)

ax2.semilogx(densities, max_sizes, 'b-', linewidth=3, marker='o', markersize=6)
ax2.axhline(np.max(np.array(avascular_viability) >= 0.8) * radii_um.max(), 
           color='red', linestyle='--', linewidth=2, label='Avascular limit')

ax2.set_xlabel('Vessel Density (vessels/cm³)')
ax2.set_ylabel('Maximum Viable Size (μm)')
ax2.set_title('Vessel Density vs Maximum Construct Size')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🩸 Vascularization Benefits:")
print("=" * 40)
avascular_max = radii_um[np.array(avascular_viability) >= 0.8].max() if np.any(np.array(avascular_viability) >= 0.8) else 0
vascular_max = max_sizes[-1]
improvement = vascular_max / avascular_max if avascular_max > 0 else float('inf')

print(f"Avascular maximum size: {avascular_max:.0f} μm")
print(f"Vascularized maximum size: {vascular_max:.0f} μm (1000 vessels/cm³)")
print(f"Size improvement: {improvement:.1f}x")
print(f"Volume improvement: {improvement**3:.1f}x")

## 🎬 Time-Dependent Growth Animation

Watch how oxygen gradients change as a spheroid grows over time.

In [ ]:
def create_growth_animation(tissue_type='Hepatocytes', final_radius_um=800, days=14):
    """
    Create animation of spheroid growth and oxygen depletion
    """
    # Growth model: exponential growth with saturation
    initial_radius_um = 50
    time_points = np.linspace(0, days, 50)
    
    # Logistic growth model
    growth_rate = 0.3  # per day
    carrying_capacity = final_radius_um
    
    radii_um = carrying_capacity / (1 + ((carrying_capacity - initial_radius_um) / initial_radius_um) * 
                                   np.exp(-growth_rate * time_points))
    
    # Calculate profiles for each time point
    profiles_data = []
    for radius_um in radii_um:
        radius_cm = radius_um / 10000
        model = SpheroidDiffusionModel(radius_cm, tissue_type)
        result = model.calculate_viability_zone()
        
        profiles_data.append({
            'radius_um': radius_um,
            'r_um': model.r * 10000,
            'C_profile': result['C_profile'],
            'viable_radius_um': result['viable_radius'] * 10000,
            'viability_fraction': result['viability_fraction']
        })
    
    # Create static plots showing key time points
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    time_indices = [0, 10, 20, 30, 40, 49]  # 6 time points
    time_labels = [f'Day {time_points[i]:.1f}' for i in time_indices]
    
    for idx, time_idx in enumerate(time_indices):
        row = idx // 3
        col = idx % 3
        ax = axes[row, col]
        
        data = profiles_data[time_idx]
        props = tissue_properties[tissue_type]
        
        # Plot concentration profile
        ax.plot(data['r_um'], data['C_profile'], 'b-', linewidth=3)
        ax.axhline(props['C_critical'], color='red', linestyle='--', 
                  linewidth=2, alpha=0.7)
        
        # Fill viable/necrotic zones
        viable_mask = data['C_profile'] >= props['C_critical']
        ax.fill_between(data['r_um'], 0, data['C_profile'], 
                       where=viable_mask, alpha=0.3, color='green')
        ax.fill_between(data['r_um'], 0, data['C_profile'], 
                       where=~viable_mask, alpha=0.3, color='red')
        
        ax.set_xlim(0, final_radius_um)
        ax.set_ylim(0, constants['C_surface'] * 1.1)
        ax.set_title(f'{time_labels[idx]}\nRadius: {data["radius_um"]:.0f} μm\n'
                    f'Viability: {data["viability_fraction"]:.1%}')
        ax.set_xlabel('Distance from center (μm)')
        ax.set_ylabel('Oxygen (mM)')
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Spheroid Growth and Oxygen Depletion - {tissue_type}', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary plot: viability over time
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Size and viability over time
    viabilities = [data['viability_fraction'] for data in profiles_data]
    
    ax1_twin = ax1.twinx()
    
    line1 = ax1.plot(time_points, radii_um, 'b-', linewidth=3, 
                     marker='o', label='Spheroid radius')
    line2 = ax1_twin.plot(time_points, viabilities, 'r-', linewidth=3, 
                         marker='s', label='Viability fraction')
    
    ax1.set_xlabel('Time (days)')
    ax1.set_ylabel('Radius (μm)', color='blue')
    ax1_twin.set_ylabel('Viability Fraction', color='red')
    ax1_twin.axhline(0.8, color='gray', linestyle='--', alpha=0.7)
    
    # Combine legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='center right')
    
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Growth Kinetics and Viability')
    
    # Plot 2: Center oxygen concentration over time
    center_oxygen = [data['C_profile'][0] for data in profiles_data]
    
    ax2.plot(time_points, center_oxygen, 'g-', linewidth=3, marker='o')
    ax2.axhline(props['C_critical'], color='red', linestyle='--', 
               linewidth=2, label=f'Critical threshold ({props["C_critical"]:.3f} mM)')
    
    ax2.set_xlabel('Time (days)')
    ax2.set_ylabel('Center Oxygen Concentration (mM)')
    ax2.set_title('Oxygen Depletion at Center')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find critical time points
    viability_80_idx = np.where(np.array(viabilities) < 0.8)[0]
    if len(viability_80_idx) > 0:
        critical_day = time_points[viability_80_idx[0]]
        critical_size = radii_um[viability_80_idx[0]]
        print(f"\n⚠️  Critical point reached:")
        print(f"Day {critical_day:.1f}: Viability drops below 80% at {critical_size:.0f} μm")
    else:
        print(f"\n✅ Viability remains above 80% throughout {days} days of growth")

# Create growth animation
print("🎬 Creating spheroid growth visualization...")
create_growth_animation('Hepatocytes', 800, 14)

## 🔬 Applications and Design Guidelines

Practical recommendations for scaffold-free biofabrication.

In [ ]:
def generate_design_guidelines():
    """
    Generate design recommendations based on diffusion analysis
    """
    guidelines = {}
    
    for tissue in tissue_properties.keys():
        # Calculate critical sizes for different applications
        critical_80 = find_critical_radius(tissue, 0.8) * 10000  # μm
        critical_90 = find_critical_radius(tissue, 0.9) * 10000  # μm
        
        # Application recommendations
        if critical_80 < 200:
            application = "Suitable for organoids, drug testing"
            scaling = "Requires vascularization for larger constructs"
        elif critical_80 < 400:
            application = "Good for tissue modeling, moderate-size constructs"
            scaling = "Consider perfusion culture or vascularization"
        else:
            application = "Excellent for large constructs, tissue engineering"
            scaling = "Can achieve clinically relevant sizes"
        
        guidelines[tissue] = {
            'critical_80': critical_80,
            'critical_90': critical_90,
            'application': application,
            'scaling': scaling,
            'metabolism': tissue_properties[tissue]['R_max'],
            'sensitivity': tissue_properties[tissue]['C_critical']
        }
    
    return guidelines

# Generate and display guidelines
guidelines = generate_design_guidelines()

print("🎯 Design Guidelines for Scaffold-Free Biofabrication")
print("=" * 70)

for tissue, guide in guidelines.items():
    print(f"\n{tissue}:")
    print(f"  Critical sizes: {guide['critical_90']:.0f} μm (90% viable), {guide['critical_80']:.0f} μm (80% viable)")
    print(f"  Applications: {guide['application']}")
    print(f"  Scaling strategy: {guide['scaling']}")

# Create comprehensive comparison
comparison_data = []
for tissue, guide in guidelines.items():
    comparison_data.append({
        'Tissue': tissue,
        'Max_Size_80%_um': guide['critical_80'],
        'Max_Size_90%_um': guide['critical_90'],
        'Oxygen_Consumption': guide['metabolism'] * 1e9,  # Convert to nM/s
        'Hypoxia_Sensitivity': guide['sensitivity']
    })

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('Max_Size_80%_um', ascending=False)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Critical sizes comparison
x = np.arange(len(df_comparison))
width = 0.35

bars1 = ax1.bar(x - width/2, df_comparison['Max_Size_90%_um'], width, 
               label='90% viable', alpha=0.8, color='lightgreen')
bars2 = ax1.bar(x + width/2, df_comparison['Max_Size_80%_um'], width, 
               label='80% viable', alpha=0.8, color='orange')

ax1.set_xlabel('Tissue Type')
ax1.set_ylabel('Critical Diameter (μm)')
ax1.set_title('Critical Sizes for Different Viability Thresholds')
ax1.set_xticks(x)
ax1.set_xticklabels(df_comparison['Tissue'], rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Oxygen consumption vs size relationship
ax2.scatter(df_comparison['Oxygen_Consumption'], df_comparison['Max_Size_80%_um'], 
           s=100, alpha=0.7, c=range(len(df_comparison)), cmap='viridis')

for i, txt in enumerate(df_comparison['Tissue']):
    ax2.annotate(txt, (df_comparison['Oxygen_Consumption'].iloc[i], 
                      df_comparison['Max_Size_80%_um'].iloc[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

ax2.set_xlabel('Oxygen Consumption Rate (nM/s)')
ax2.set_ylabel('Critical Size at 80% Viability (μm)')
ax2.set_title('Metabolism vs Maximum Achievable Size')
ax2.grid(True, alpha=0.3)

# Plot 3: Hypoxia sensitivity analysis
ax3.scatter(df_comparison['Hypoxia_Sensitivity'], df_comparison['Max_Size_80%_um'], 
           s=100, alpha=0.7, c=range(len(df_comparison)), cmap='plasma')

for i, txt in enumerate(df_comparison['Tissue']):
    ax3.annotate(txt, (df_comparison['Hypoxia_Sensitivity'].iloc[i], 
                      df_comparison['Max_Size_80%_um'].iloc[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

ax3.set_xlabel('Critical Oxygen Threshold (mM)')
ax3.set_ylabel('Critical Size at 80% Viability (μm)')
ax3.set_title('Hypoxia Sensitivity vs Maximum Size')
ax3.grid(True, alpha=0.3)

# Plot 4: Application suitability matrix
applications = ['Organoids\n(<200 μm)', 'Tissue Models\n(200-400 μm)', 
               'Large Constructs\n(>400 μm)']
suitability_matrix = np.zeros((len(df_comparison), 3))

for i, size in enumerate(df_comparison['Max_Size_80%_um']):
    if size >= 400:
        suitability_matrix[i] = [1, 1, 1]  # Suitable for all
    elif size >= 200:
        suitability_matrix[i] = [1, 1, 0]  # Good for organoids and models
    else:
        suitability_matrix[i] = [1, 0, 0]  # Only organoids

im = ax4.imshow(suitability_matrix, cmap='RdYlGn', aspect='auto')
ax4.set_xticks(range(3))
ax4.set_xticklabels(applications)
ax4.set_yticks(range(len(df_comparison)))
ax4.set_yticklabels(df_comparison['Tissue'])
ax4.set_title('Application Suitability Matrix')

# Add text annotations
for i in range(len(df_comparison)):
    for j in range(3):
        text = '✓' if suitability_matrix[i, j] == 1 else '✗'
        ax4.text(j, i, text, ha='center', va='center', 
                color='white' if suitability_matrix[i, j] == 0 else 'black',
                fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Key Design Principles:")
print("=" * 30)
print("1. High-metabolism cells (hepatocytes, cardiomyocytes) require smaller constructs")
print("2. Hypoxia-tolerant cells (chondrocytes) can form larger avascular constructs")
print("3. Vascularization is essential for constructs >500 μm with most cell types")
print("4. Perfusion culture can extend viable sizes by 2-5x")
print("5. Consider cell type when selecting spheroid formation methods")

## 📝 Student Tasks and Reflection

Complete these tasks to deepen your understanding of nutrient diffusion limitations.

### Task 1: Critical Size Prediction

**Question:** Based on your analysis, predict the maximum viable size for a new cell type with these properties:
- R_max = 1.2 × 10⁻⁸ mM/s
- C_critical = 0.012 mM
- K_m = 0.011 mM

Use the code below to test your prediction:

In [ ]:
# Task 1: Add new cell type and analyze
tissue_properties['Custom_cells'] = {
    'R_max': 1.2e-8,
    'K_m': 0.011,
    'C_critical': 0.012,
    'cell_density': 1e8,
    'description': 'Student-defined custom cell type'
}

# Calculate critical size
custom_critical = find_critical_radius('Custom_cells', 0.8) * 10000
print(f"Critical size for custom cells: {custom_critical:.0f} μm")

# Compare with existing cell types
print("\nComparison with existing types:")
for tissue in ['Hepatocytes', 'Chondrocytes', 'Tumor_cells']:
    existing_critical = find_critical_radius(tissue, 0.8) * 10000
    print(f"{tissue}: {existing_critical:.0f} μm")

# Your prediction vs actual result
print("\n📊 Write your prediction and compare with the calculated result:")
print("My prediction: ___ μm")
print(f"Actual result: {custom_critical:.0f} μm")
print("Difference: ___ μm")
print("\nExplanation for any differences:")
print("___________________________________")

### Task 2: Vascularization Strategy

**Scenario:** You need to create a 2 mm diameter hepatocyte construct for drug testing.

**Questions:**
1. Is this feasible without vascularization?
2. What vessel density would you need?
3. How would you implement this experimentally?

Analyze using the code below:

In [ ]:
# Task 2: Vascularization analysis
target_size_um = 2000  # 2 mm diameter
target_radius_cm = target_size_um / 20000  # Convert to radius in cm
tissue_type = 'Hepatocytes'

# Check avascular viability
avascular_model = SpheroidDiffusionModel(target_radius_cm, tissue_type)
avascular_result = avascular_model.calculate_viability_zone()

print(f"🎯 Target: {target_size_um} μm diameter {tissue_type} construct")
print("=" * 50)
print(f"Avascular viability: {avascular_result['viability_fraction']:.1%}")
print(f"Necrotic core size: {(target_size_um/2 - avascular_result['viable_radius']*10000)*2:.0f} μm")

if avascular_result['viability_fraction'] < 0.8:
    print("\n❌ Not feasible without vascularization!")
    
    # Test different vessel densities
    print("\n🩸 Testing vascularization strategies:")
    test_densities = [10, 50, 100, 200, 500, 1000]
    
    for density in test_densities:
        vasc_result = simulate_vascularized_tissue(target_radius_cm, tissue_type, density)
        viability = vasc_result['viability_fraction']
        status = "✅" if viability >= 0.8 else "❌"
        print(f"{status} {density:4d} vessels/cm³: {viability:.1%} viable")
        
        if viability >= 0.8:
            spacing_um = vasc_result['vessel_spacing'] * 10000
            print(f"   → Required vessel spacing: {spacing_um:.0f} μm")
            break
else:
    print("\n✅ Feasible without vascularization!")

print("\n📝 Your analysis:")
print("1. Feasible without vascularization? Yes/No: ___")
print("2. Minimum vessel density needed: ___ vessels/cm³")
print("3. Experimental implementation strategy:")
print("   - Method 1: ________________________________")
print("   - Method 2: ________________________________")
print("   - Method 3: ________________________________")

### Task 3: Method Selection

**Challenge:** Choose the best spheroid formation method for different applications based on diffusion limitations.

Consider the methods from Chapter 2.1.5:
- Hanging drop
- Liquid overlay
- Spinner flask
- Microcarrier beads
- Magnetic levitation
- Emulsion

In [ ]:
# Task 3: Method selection analysis
applications = {
    'Drug_screening_organoids': {
        'target_size_um': 200,
        'cell_type': 'Hepatocytes',
        'throughput': 'High',
        'uniformity': 'Critical',
        'cost': 'Low'
    },
    'Cartilage_tissue_engineering': {
        'target_size_um': 800,
        'cell_type': 'Chondrocytes',
        'throughput': 'Medium',
        'uniformity': 'Important',
        'cost': 'Medium'
    },
    'Cancer_research_models': {
        'target_size_um': 500,
        'cell_type': 'Tumor_cells',
        'throughput': 'High',
        'uniformity': 'Important',
        'cost': 'Low'
    }
}

print("🎯 Method Selection Challenge")
print("=" * 40)

for app_name, requirements in applications.items():
    print(f"\n{app_name.replace('_', ' ').title()}:")
    
    # Check if target size is viable
    radius_cm = requirements['target_size_um'] / 20000
    model = SpheroidDiffusionModel(radius_cm, requirements['cell_type'])
    result = model.calculate_viability_zone()
    
    print(f"  Target: {requirements['target_size_um']} μm {requirements['cell_type']}")
    print(f"  Predicted viability: {result['viability_fraction']:.1%}")
    print(f"  Requirements: {requirements['throughput']} throughput, {requirements['uniformity']} uniformity")
    
    if result['viability_fraction'] < 0.8:
        print(f"  ⚠️  Size may be too large - consider reducing to {result['viable_radius']*20000:.0f} μm")
    else:
        print(f"  ✅ Size is viable for this cell type")
    
    print(f"  Your method choice: ________________")
    print(f"  Justification: _____________________")

print("\n\n📋 Method Comparison Matrix:")
print("""
Method             | Throughput | Uniformity | Cost | Size Control | Notes
-------------------|------------|------------|------|--------------|-------
Hanging drop       | Low        | High       | Low  | Excellent    | Best uniformity
Liquid overlay     | High       | Medium     | Low  | Good         | Scalable
Spinner flask      | High       | Low        | Med  | Variable     | Good mixing
Microcarrier       | High       | Medium     | Med  | Good         | Industrial scale
Magnetic levitation| Medium     | High       | High | Excellent    | Precise control
Emulsion           | Very High  | High       | Med  | Excellent    | Microfluidics
""")

print("\n🤔 Reflection Questions:")
print("1. How does diffusion limitation influence method selection?")
print("   Answer: ___________________________________________")
print("\n2. Which cell types are most challenging for large constructs?")
print("   Answer: ___________________________________________")
print("\n3. What strategies could overcome size limitations?")
print("   Answer: ___________________________________________")

## 🎯 Key Takeaways and Future Directions

### What We Learned:

1. **Diffusion limits scaffold-free construct size** - Most cell types cannot survive in constructs >500 μm without vascularization

2. **Cell type matters enormously** - High-metabolism cells (hepatocytes) are most limited, while hypoxia-tolerant cells (chondrocytes) can form larger constructs

3. **Necrotic cores form predictably** - Mathematical models can predict when and where cell death will occur

4. **Vascularization is transformative** - Even modest vessel density can increase viable size by orders of magnitude

5. **Design guidelines are tissue-specific** - One size does not fit all applications

### Future Directions:

- **Advanced vascularization strategies** (bioprinted vessels, angiogenic factors)
- **Oxygen-carrying biomaterials** (perfluorocarbon emulsions, hemoglobin-based carriers)
- **Metabolic engineering** (reducing cellular oxygen consumption)
- **Perfusion bioreactors** (external oxygenation and nutrient delivery)
- **Hybrid approaches** (combining multiple strategies)

### Connection to Chapter 2:

This exercise demonstrates why:
- **Spheroid size control** is critical for viability
- **Bioreactor design** must consider mass transport
- **ECM engineering** needs to support vascularization
- **Smart materials** could respond to oxygen gradients

---

### 🏆 Congratulations!

You've completed Exercise 5 and explored the fundamental physics that limits scaffold-free biofabrication. These insights will guide your future work in tissue engineering and help you design more effective biofabrication strategies.

**Next steps:**
- Apply these principles to your own research projects
- Explore advanced vascularization strategies
- Consider how different bioreactor designs address mass transport
- Connect diffusion limitations to clinical translation challenges